In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# Pandas Options
pd.options.display.float_format = '{:.2f}'.format

# 1. Carga de datos

Se crea una dataframe con la información de los datos dados, se parsea la fecha y se agrega el identificador de los clientes.

In [ ]:
# Cargar datos

file_path = '../data/raw/Datos.xlsx'
excel_file = pd.ExcelFile(file_path)

client_dfs = []
for cliente in excel_file.sheet_names:
    
    df = pd.read_excel(excel_file, sheet_name=cliente)
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Cliente'] = cliente
    client_dfs.append(df)

excel_file.close()

# Concatener datos
data = pd.concat(client_dfs, ignore_index=True)

# Guardar datos completos
data.to_csv("../data/raw/data.csv", index=False)

data.head()

In [ ]:
# Descripción de los datos

# Estadísticas por cliente
resumen_clientes = data.groupby('Cliente').agg(
    fecha_min=('Fecha', 'min'),
    fecha_max=('Fecha', 'max'),
    
    presion_max=('Presion', 'max'),
    presion_min=('Presion', 'min'),
    presion_mean=('Presion', 'mean'),
    presion_std=('Presion', 'std'),
    
    temperatura_max=('Temperatura', 'max'),
    temperatura_min=('Temperatura', 'min'),
    temperatura_mean=('Temperatura', 'mean'),
    temperatura_std=('Temperatura', 'std'),
    
    volumen_max=('Volumen', 'max'),
    volumen_min=('Volumen', 'min'),
    volumen_mean=('Volumen', 'mean'),
    volumen_std=('Volumen', 'std'),
    
    num_registros=('Volumen', 'count')
).reset_index()

# Número total de clientes únicos
num_clientes = data['Cliente'].nunique()

print(f"Total de clientes: {num_clientes}")
resumen_clientes

De la descripción de los datos se tienen que las fechas entre el 14 de enero de 2019 y el 31 de diciembre de 2023.

In [ ]:
# Gráfica de comportamiento por clientes. 
# Promedio movil & Rangos Intercuartilicos IQR

# Lista de clientes únicos
clientes = data['Cliente'].unique()

ncols = 4
nrows = math.ceil(num_clientes / ncols)

# Crear subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 4, nrows * 2), sharex=False)
axes = axes.flatten()

ventana = 168 # 7 días x 24 horas
k = 2

# Recorrer clientes y sus subplots
for i, cliente in enumerate(clientes):
    ax = axes[i]
    data_cliente = data[data['Cliente'] == cliente].sort_values('Fecha').copy()
    
    # Cálculo de rolling stats
    data_cliente['volumen_promedio'] = data_cliente['Volumen'].rolling(window=ventana).mean()
    
    # Cálculo de percentiles
    data_cliente['Q1'] = data_cliente['Volumen'].rolling(window=ventana).quantile(0.25)
    data_cliente['Q3'] = data_cliente['Volumen'].rolling(window=ventana).quantile(0.75)
    data_cliente['IQR'] = data_cliente['Q3'] - data_cliente['Q1']
    
    # Límites intercuartílicos
    data_cliente['IQR_inf'] = (data_cliente['Q1'] - 1.5 * data_cliente['IQR']).clip(lower=0)
    data_cliente['IQR_sup'] = data_cliente['Q3'] + 1.5 * data_cliente['IQR']

    # Graficar cada cliente
    ax.scatter(data_cliente['Fecha'], data_cliente['Volumen'], alpha=1, s=1)
    ax.plot(data_cliente['Fecha'], data_cliente['volumen_promedio'], color='red', linewidth=1,)
    ax.plot(data_cliente['Fecha'], data_cliente['IQR_sup'], color='green', linestyle='--', linewidth=0.8)
    ax.plot(data_cliente['Fecha'], data_cliente['IQR_inf'], color='green', linestyle='--', linewidth=0.8)
    
    ax.set_title(cliente, fontsize=9, loc='left')

# Apagar subplots vacíos 
for j in range(len(clientes), len(axes)):
    fig.delaxes(axes[j]) 

plt.tight_layout()
plt.show()

In [ ]:
cliente = 'CLIENTE12' 

data_cliente = data[data['Cliente'] == cliente].sort_values('Fecha').copy()
    
# Cálculo de rolling stats
data_cliente['volumen_promedio'] = data_cliente['Volumen'].rolling(window=ventana).mean()

# Cálculo de percentiles
data_cliente['Q1'] = data_cliente['Volumen'].rolling(window=ventana).quantile(0.25)
data_cliente['Q3'] = data_cliente['Volumen'].rolling(window=ventana).quantile(0.75)
data_cliente['IQR'] = data_cliente['Q3'] - data_cliente['Q1']

# Límites intercuartílicos
data_cliente['IQR_inf'] = (data_cliente['Q1'] - 1.5 * data_cliente['IQR']).clip(lower=0)
data_cliente['IQR_sup'] = data_cliente['Q3'] + 1.5 * data_cliente['IQR']

# Graficar cada cliente
plt.scatter(data_cliente['Fecha'], data_cliente['Volumen'], alpha=1, s=1)
plt.plot(data_cliente['Fecha'], data_cliente['volumen_promedio'], color='red', linewidth=1,)
plt.plot(data_cliente['Fecha'], data_cliente['IQR_sup'], color='green', linestyle='--', linewidth=0.8)
plt.plot(data_cliente['Fecha'], data_cliente['IQR_inf'], color='green', linestyle='--', linewidth=0.8)

plt.title(f'Consumo de gas - {cliente} con Promedio Móvil Semanal')
plt.xlabel('Fecha')
plt.ylabel('Volumen')
plt.tight_layout()
plt.show()